In [100]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
# Show all output for a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [102]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [103]:
from src.inference import get_feature_store

In [104]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*35)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.hour.between(fetch_data_from, fetch_data_to)]

2025-05-11 12:12:25,476 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:12:25,507 INFO: Initializing external client
2025-05-11 12:12:25,507 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:12:26,125 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-06 16:12:25.475652+00:00 to 2025-05-11 15:12:25.475652+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


In [105]:
ts_data.sort_values(["start_station_id", "hour"]).reset_index(drop=True)

,hour,start_station_id,ride_count
0,2025-04-12 10:00:00+00:00,5329.03,12
1,2025-04-12 11:00:00+00:00,5329.03,22
2,2025-04-12 12:00:00+00:00,5329.03,18
3,2025-04-12 13:00:00+00:00,5329.03,18
4,2025-04-12 14:00:00+00:00,5329.03,13
...,...,...,...
2958,2025-05-11 11:00:00+00:00,6140.05,16
2959,2025-05-11 12:00:00+00:00,6140.05,7
2960,2025-05-11 13:00:00+00:00,6140.05,20
2961,2025-05-11 14:00:00+00:00,6140.05,27


In [106]:
ts_data["hour"] = ts_data["hour"].dt.tz_localize(None)

In [107]:
from src.data_utils import transform_ts_data_into_features
features = transform_ts_data_into_features(ts_data, window_size=24*28, step_size=23)

Skipping station 5788.13: Not enough data to create even one window.


In [108]:
features.head()

,ride_count_t-672,ride_count_t-671,ride_count_t-670,ride_count_t-669,ride_count_t-668,ride_count_t-667,ride_count_t-666,ride_count_t-665,ride_count_t-664,ride_count_t-663,...,ride_count_t-8,ride_count_t-7,ride_count_t-6,ride_count_t-5,ride_count_t-4,ride_count_t-3,ride_count_t-2,ride_count_t-1,start_station_id,hour
0,20,0,0,13,3,18,2,14,6,8,...,37,30,26,8,6,24,20,1,5905.14,2025-05-05 05:00:00
1,3,8,0,0,8,52,20,4,31,4,...,30,2,0,9,0,0,26,24,5905.14,2025-04-25 08:00:00
2,4,3,32,1,19,0,5,14,1,0,...,7,18,6,5,15,26,3,3,5905.14,2025-04-08 06:00:00
3,35,0,11,9,39,32,34,0,38,0,...,16,19,0,24,14,20,0,19,5905.14,2025-04-10 07:00:00
4,16,38,1,3,28,37,0,37,7,3,...,10,15,1,1,39,0,9,17,5905.14,2025-04-11 18:00:00


In [109]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-11 12:12:29,947 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:12:29,953 INFO: Initializing external client
2025-05-11 12:12:29,953 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:12:30,460 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-12 16:12:29.946969+00:00 to 2025-05-11 15:12:29.946969+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.82s) 
Skipping station 5788.13: Not enough data to create even one window.


In [110]:
features.shape

(6, 674)

In [111]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-11 12:12:37,137 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-05-11 12:12:37,160 INFO: Initializing external client
2025-05-11 12:12:37,161 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:12:37,595 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


Downloading: 0.000%|          | 0/325454 elapsed<00:00 remaining<?

In [112]:
features.shape

(6, 674)

In [113]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [114]:
predictions

,start_station_id,predicted_demand
0,5329.03,35.0
1,5329.03,24.0
2,5905.14,1.0
3,5905.14,31.0
4,6140.05,31.0
5,6140.05,33.0
